# Sales Category Prediction - Final Model Optimization

In [ ]:
# imports
import numpy as np
import pandas as pd
import gc
import os
import re
import time
import psutil
import logging
import warnings
from datetime import datetime, timedelta
from tqdm.notebook import tqdm, trange
from glob import glob
import scipy.sparse

# Feature processing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import category_encoders as ce

# Models
from lightgbm import LGBMRegressor

# Hyperparameter optimization
import optuna
from optuna.samplers import TPESampler

# Feature importance
import shap

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 500)

## Configuration

In [ ]:
CONFIG = {
    # Paths
    "splits_dir": "../data/splits/",
    "weather_path": "../data/weather_new.csv",
    "artifacts_dir": "../artifacts/weather_simple/",
    
    # Preprocessing
    "chunk_months": 1,  # Size of sliding window in months (1 or 2)
    "drop_columns": {"group", "year", "unit"},
    
    # Models
    "target_column": "qnt",
    "date_column": "calday",
    
    # Random seed
    "random_seed": 42,
    
    # Optuna optimization settings
    "n_trials": 50,        # Reduced from 50 to speed up optimization
    "use_pruning": True,   # Enable pruning for faster convergence
    "early_stop": 50,     # Aggressive early stopping for trials
    "timeout_per_trial": 1800
}

# Create artifacts directory if it doesn't exist
os.makedirs(CONFIG["artifacts_dir"], exist_ok=True)

In [ ]:
# Feature categories
BOOL_EXPLICIT = {"bu_exists", "freezing_day", "cold_day", "warm_day", "hot_day"}
BOOL_PATTERNS = [r"^is_", r"^has_", r"_had_high_", r"_had_low_", r"_exists$"]

LOW_EXPLICIT = {
    "matrix_type", "country_id", "format_merch", "geolocal_type",
    "season", "type_bonus_id", "seasonal_group",
    "category_major", "category_detailed", "category_full",
    "preciptype", "conditions"
}
LOW_PATTERNS = [r"^category_"]

MED_EXPLICIT = {
    "brand_id", "index_material", "index_store", "type_for_customer",
    "week_iso", "day_of_week", "day_of_month", "month", "quarter", "source_month"
}

NUM_PATTERNS = [
    r"^(temp|min|max|tempmax|tempmin|feelslike[a-z]*|dew|humidity|precip|snow|snowdepth"
    r"|wind(gust|speed|dir)|sealevelpressure|cloudcover|visibility"
    r"|solarradiation|solarenergy|uvindex|moonphase|daylight_hours"
    r"|heat_index|temp_range)",
    r".*_lag_\\d+d$", r".*_d\\d+to\\d+_(mean|min|max|std)$"
]

# Define weather feature patterns for filtering baseline model - much more comprehensive list
WEATHER_PATTERNS = [
    # Base weather measurements
    # r"temp", r"feelslike", r"dew", r"humidity", r"precip", 
    # r"snow", r"wind", r"sealevelpressure", r"cloudcover", r"visibility", 
    # r"solar", r"uvindex", r"moonphase", r"daylight", r"heat_index",
    
    # # Weather condition indicators
    # r"precipprob", r"precipcover", r"snowdepth",
    
    # # Boolean weather flags
    # r"^has_precipitation", r"^has_rain", r"^has_snow",
    # r"^is_clear", r"^is_overcast", r"^is_rain", r"^is_snow", 
    # r"^is_fog", r"^is_cloudy", r"^is_partially_cloudy",
    
    # # Temperature categories
    # r"temp_range", r"freezing_day", r"cold_day", r"warm_day", r"hot_day"
]

## Utility Functions

In [ ]:
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024

def log_step(step_name):
    """Log step name and memory usage"""
    mem_mb = get_memory_usage()
    logging.info(f"{step_name}: {mem_mb:.2f} MB")
    return mem_mb

# Evaluation metrics
def calculate_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def calculate_mape(y_true, y_pred):
    mask_nonzero = (y_true != 0) & (y_pred != 0)
    mask_one_zero = ((y_true == 0) & (y_pred != 0)) | ((y_true != 0) & (y_pred == 0))
    
    mape_nonzero = np.abs((y_true[mask_nonzero] - y_pred[mask_nonzero]) / y_true[mask_nonzero])
    mape_one_zero = np.ones(mask_one_zero.sum())
    
    if len(mape_nonzero) + len(mape_one_zero) == 0:
        return 0
    
    total_mape = np.concatenate([mape_nonzero, mape_one_zero]) if len(mape_one_zero) > 0 and len(mape_nonzero) > 0 else \
                (mape_nonzero if len(mape_nonzero) > 0 else mape_one_zero)
    return np.mean(total_mape)

def calculate_smape(y_true, y_pred):
    mask_nonzero = (y_true != 0) & (y_pred != 0)
    mask_one_zero = ((y_true == 0) & (y_pred != 0)) | ((y_true != 0) & (y_pred == 0))
    
    smape_nonzero = 2 * np.abs(y_true[mask_nonzero] - y_pred[mask_nonzero]) / (np.abs(y_true[mask_nonzero]) + np.abs(y_pred[mask_nonzero]))
    smape_one_zero = np.ones(mask_one_zero.sum())
    
    if len(smape_nonzero) + len(smape_one_zero) == 0:
        return 0
    
    total_smape = np.concatenate([smape_nonzero, smape_one_zero]) if len(smape_one_zero) > 0 and len(smape_nonzero) > 0 else \
                 (smape_nonzero if len(smape_nonzero) > 0 else smape_one_zero)
    return np.mean(total_smape)

# Additional metrics
def calculate_rmse(y_true, y_pred):
    """Calculate Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def calculate_r2(y_true, y_pred):
    """Calculate R-squared (coefficient of determination)"""
    from sklearn.metrics import r2_score
    return r2_score(y_true, y_pred)

def calculate_wape(y_true, y_pred):
    """Calculate Weighted Absolute Percentage Error"""
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

def calculate_bias(y_true, y_pred):
    """Calculate bias (Mean Percent Forecast Error)
    >0 → over-forecast (over-stock)
    <0 → under-forecast (under-stock)"""
    mask = y_true != 0
    if not np.any(mask):
        return 0
    
    percent_errors = (y_pred[mask] - y_true[mask]) / y_true[mask]
    return np.mean(percent_errors)

def calculate_stock_stats(y_true, y_pred, action_price):
    """Calculate over-stock and under-stock statistics"""
    differences = y_pred - y_true
    over_stock = np.where(differences > 0, differences * action_price, 0)
    under_stock = np.where(differences < 0, -differences * action_price, 0)
    
    total_sales = np.sum(y_true * action_price)
    over_stock_sum = np.sum(over_stock)
    under_stock_sum = np.sum(under_stock)
    
    # Avoid division by zero
    if total_sales == 0:
        over_stock_percent = 0
        under_stock_percent = 0
    else:
        over_stock_percent = over_stock_sum / total_sales * 100
        under_stock_percent = under_stock_sum / total_sales * 100
    
    return {
        'total_sales': total_sales,
        'over_stock': over_stock_sum,
        'under_stock': under_stock_sum,
        'over_stock_percent': over_stock_percent,
        'under_stock_percent': under_stock_percent
    }

In [ ]:
def postprocess_predictions(y_pred, qnt_max):
    """Apply post-processing rules to prediction"""
    # Clip negative values to 0
    y_pred = np.clip(y_pred, 0, None)
    
    # Apply outlier correction where needed
    mask = (qnt_max >= 5) & (y_pred > 2 * qnt_max)
    if np.any(mask):
        y_pred[mask] = 2 * qnt_max[mask]
    
    # Round to nearest integer
    y_pred = np.round(y_pred).astype(int)
    
    return y_pred

## Data Preprocessing

In [ ]:
def get_feature_types(df):
    """Identify feature types based on rules"""
    columns = df.columns.tolist()
    feature_types = {'bool_cols': [], 'low_cat_cols': [], 'med_cat_cols': [], 'num_cols': []}
    
    # Find target and ID columns
    target_col = CONFIG['target_column']
    date_col = CONFIG['date_column']
    drop_cols = list(CONFIG['drop_columns']) + [target_col, date_col] if target_col in columns and date_col in columns else []
    
    # Collect columns by type
    for col in columns:
        if col in drop_cols:
            continue
        
        # Check bool columns
        if col in BOOL_EXPLICIT or any(re.match(pattern, col) for pattern in BOOL_PATTERNS) or df[col].dtype == bool:
            feature_types['bool_cols'].append(col)
            continue
            
        # Check low cardinality categorical
        if col in LOW_EXPLICIT or any(re.match(pattern, col) for pattern in LOW_PATTERNS):
            feature_types['low_cat_cols'].append(col)
            continue
            
        # Check medium cardinality categorical
        if col in MED_EXPLICIT:
            feature_types['med_cat_cols'].append(col)
            continue
            
        # Check numeric by pattern
        if any(re.match(pattern, col) for pattern in NUM_PATTERNS) or df[col].dtype in ['int32', 'int64', 'float32', 'float64']:
            feature_types['num_cols'].append(col)
    
    return feature_types

def create_column_transformer(df):
    """Create a ColumnTransformer for preprocessing"""
    feature_types = get_feature_types(df)
    
    # Log the number of features of each type for debugging
    logging.info(f"Feature counts: bool={len(feature_types['bool_cols'])}, low_cat={len(feature_types['low_cat_cols'])}, "
                 f"med_cat={len(feature_types['med_cat_cols'])}, num={len(feature_types['num_cols'])}")
    
    transformers = []
    
    # Boolean columns to uint8
    if feature_types['bool_cols']:
        transformers.append(('bool', 'passthrough', feature_types['bool_cols']))
    
    # Low cardinality categorical columns to one-hot
    if feature_types['low_cat_cols']:
        transformers.append(('low_cat', 
                            OneHotEncoder(sparse_output=True, handle_unknown='ignore', dtype=np.int8),
                            feature_types['low_cat_cols']))
    
    # Medium cardinality categorical columns with count encoding
    if feature_types['med_cat_cols']:
        # First check if we have any medium cardinality columns
        if len(feature_types['med_cat_cols']) > 0:
            # Create a copy of the DataFrame with only the medium cardinality columns
            med_cat_df = df[feature_types['med_cat_cols']].copy()
            
            # Convert all columns to categorical type
            for col in med_cat_df.columns:
                med_cat_df[col] = med_cat_df[col].astype('category')
            transformers.append(('med_cat', 
                              ce.CountEncoder(normalize=True), 
                              feature_types['med_cat_cols']))
    
    # Numeric columns - just pass through, no imputation needed
    if feature_types['num_cols']:
        transformers.append(('num', 
                            'passthrough', 
                            feature_types['num_cols']))
    
    return ColumnTransformer(transformers, remainder='drop', n_jobs=1)  # Use single thread to reduce memory

def cast_types(df):
    """Downcast types for efficiency"""
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'bool':
            df[col] = df[col].astype('uint8')
    return df
    
def load_and_preprocess_data(split_file):
    """Load and preprocess a split file"""
    log_step(f"Loading split file {os.path.basename(split_file)}")
    
    # Read the split file
    df = pd.read_csv(split_file)
    
    # Read weather data
    weather_df = pd.read_csv(CONFIG['weather_path'])
    
    # Merge with weather data
    df = pd.merge(df, weather_df, on=CONFIG['date_column'], how='left')
    
    # Cast types
    df = cast_types(df)
    
    # Drop unnecessary columns
    for col in CONFIG['drop_columns']:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
    
    # Sort by date
    df.sort_values(by=CONFIG['date_column'], inplace=True)
    
    # Convert date to datetime
    if df[CONFIG['date_column']].dtype == 'object':
        df[CONFIG['date_column']] = pd.to_datetime(df[CONFIG['date_column']])
    
    # Explicitly convert medium cardinality columns to category without logging each conversion
    for col in MED_EXPLICIT:
        if col in df.columns:
            df[col] = df[col].astype('category')
    
    # Calculate max_qnt for post-processing only if it doesn't exist
    if 'qnt_max' not in df.columns:
        logging.info(f"'qnt_max' column not found, calculating it now")
        df['qnt_max'] = df.groupby(['brand_id', 'country_id', 'category_detailed'])[CONFIG['target_column']].transform('max')
    
    # Log transformation for specific quantity columns
    log_columns = [
        'qnt', 'qnt_loss', 'qnt_lag_14d', 'qnt_lag_21d', 'qnt_lag_28d', 
        'qnt_lag_avg', 'qnt_max', 'qnt_min', 'qnt_mean', 'qnt_median'
    ]
    
    # Store original values before transformation
    for col in log_columns:
        if col in df.columns:
            df[f"{col}_orig"] = df[col].copy()
    
    # Apply log1p transformation
    for col in log_columns:
        if col in df.columns:
            df[col] = np.log1p(df[col])
    
    log_step(f"Preprocessed split shape: {df.shape}")
    return df

def filter_weather_features(feature_cols):
    """Filter out weather-related features for baseline model"""
    baseline_features = []
    weather_features = []

    WEATHER_COLUMNS = [
        "tempmax", "tempmin", "temp", "feelslikemax", "feelslikemin", "feelslike",
        "dew", "humidity", "precip", "precipprob", "precipcover", "snow", "snowdepth",
        "windgust", "windspeed", "winddir", "sealevelpressure", "cloudcover", "visibility",
        "solarradiation", "solarenergy", "uvindex", "moonphase", "daylight_hours"
    ]
    
    for col in feature_cols:
        if col in WEATHER_COLUMNS:
            weather_features.append(col)
        else:
            baseline_features.append(col)
            
    logging.info(f"Total features: {len(feature_cols)}, Baseline features: {len(baseline_features)}, Weather features: {len(weather_features)}")
    
    # Removed detailed weather feature logging that was too verbose
    
    return baseline_features, weather_features

## Hyperparameter Optimization with Optuna

In [ ]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    """Define the objective function for Optuna"""
    param = {
        # Removed 'objective' parameter since it's the default
        'metric': 'rmse',  # Changed from 'mae' to 'rmse' which is better for log-transformed data
        'device_type': 'gpu',
        'boosting_type': 'gbdt',
        'gpu_platform_id': 1,
        'gpu_device_id': 0,
        'n_jobs': -1,
        'random_state': CONFIG["random_seed"],
        'n_estimators': 500,
        'verbose': -1,
                    
        # Hyperparameters to optimize
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 63, 255),
        'max_depth': trial.suggest_int('max_depth', 6, 18),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 250),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-5, 5, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 2.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 3.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'max_bin': trial.suggest_categorical('max_bin', [255]),
    }
    
    # Initialize LightGBM model with the trial parameters
    model = LGBMRegressor(**param)
    
    # Import lightgbm for callbacks
    import lightgbm as lgb
    
    # Custom callback for Hyperband pruning
    pruning_callback = optuna.integration.LightGBMPruningCallback(
        trial, 'rmse', valid_name='valid_0'
    )
    
    # Train the model with early stopping and pruning callback
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=CONFIG["early_stop"]),
            lgb.log_evaluation(period=100),
            pruning_callback  # Add Optuna pruning callback
        ]
    )
    
    # Explicitly release memory
    result = model.best_score_['valid_0']['rmse']
    del model
    gc.collect()
    
    # Return the best validation score
    return result

In [ ]:
def optimize_hyperparameters(X_train, y_train, X_valid, y_valid, split_name):
    """Run Optuna optimization to find best hyperparameters"""
    log_step(f"Starting hyperparameter optimization for {split_name}")
    
    # Define path for saved hyperparameters
    params_file = f"{CONFIG['artifacts_dir']}best_params_{split_name}.json"
    
    # Check if we already have saved parameters
    if os.path.exists(params_file):
        import json
        logging.info(f"Found saved hyperparameters for {split_name}, loading from {params_file}")
        with open(params_file, 'r') as f:
            loaded_params = json.load(f)
            
        # Convert parameters to the correct types
        best_params = {}
        
        # Integer parameters
        int_params = ['num_leaves', 'max_depth', 'min_child_samples']
        for param in int_params:
            if param in loaded_params:
                best_params[param] = int(loaded_params[param])
        
        # Float parameters
        float_params = ['learning_rate', 'subsample', 'colsample_bytree', 'reg_alpha', 'reg_lambda']
        for param in float_params:
            if param in loaded_params:
                best_params[param] = float(loaded_params[param])
        
        # Add fixed parameters that may not be saved in the JSON
        best_params.update({
            # Removed 'objective': 'regression'
            'metric': 'rmse',  # Changed from 'mae' to 'rmse'
            'boosting_type': 'gbdt',
            'device_type': 'gpu',
            'gpu_platform_id': 1,
            'gpu_device_id': 0,
            'n_jobs': 1,
            'random_state': CONFIG["random_seed"],
            'n_estimators': 20000,  # Will use early stopping
            'verbose': -1,  # Suppress verbose output
        })
        
        logging.info(f"Loaded hyperparameters: {best_params}")
        return best_params
    
    # Use Hyperband pruner for more efficient hyperparameter search
    from optuna.pruners import HyperbandPruner
    
    # Hyperband pruner is more efficient than MedianPruner for tree-based models
    # It allocates resources adaptively, terminating unpromising trials early
    hyperband_pruner = HyperbandPruner(
        min_resource=10,     # Minimum number of iterations before pruning
        max_resource=500,    # Maximum iterations per trial
        reduction_factor=3    # Reduction factor for successive halving
    )
    
    # Use a more sophisticated TPE sampler with Hyperband pruner
    sampler = TPESampler(
        seed=CONFIG["random_seed"],
        n_startup_trials=10,   
        multivariate=True,     # Consider parameter correlations
        consider_prior=True,   # Use prior distributions
        consider_magic_clip=True,
        consider_endpoints=True
    )
    
    study = optuna.create_study(
        direction='minimize',  # Still minimize for RMSE
        sampler=sampler,
        pruner=hyperband_pruner,  # Use Hyperband pruner instead of MedianPruner
        study_name=f"lgbm_optimization_{split_name}"
    )
    
    # Ensure we're optimizing on a representative sample
    logging.info(f"Optimizing hyperparameters using full feature set (including weather)")
    
    if X_train.shape[0] > 200000:
        sample_indices = np.random.choice(X_train.shape[0], 200000, replace=False)
        X_train_sample = X_train[sample_indices]
        y_train_sample = y_train.iloc[sample_indices] if isinstance(y_train, pd.Series) else y_train[sample_indices]
        
        # Run optimization with progress bar on sampled data
        study.optimize(
            lambda trial: objective(trial, X_train_sample, y_train_sample, X_valid, y_valid),
            n_trials=CONFIG["n_trials"],
            callbacks=[
                lambda study, trial: tqdm.write(f"Trial {trial.number} finished with value: {trial.value}"),
                # Add explicit GPU memory cleanup after each trial
                lambda study, trial: gc.collect()
            ],
            show_progress_bar=True,
            n_jobs=1  # Use sequential processing for GPU tasks
        )
    else:
        # Run optimization with progress bar on full dataset for smaller datasets
        study.optimize(
            lambda trial: objective(trial, X_train, y_train, X_valid, y_valid),
            n_trials=CONFIG["n_trials"],
            callbacks=[
                lambda study, trial: tqdm.write(f"Trial {trial.number} finished with value: {trial.value}"),
                # Add explicit GPU memory cleanup after each trial
                lambda study, trial: gc.collect()
            ],
            show_progress_bar=True,
            n_jobs=1  # Use sequential processing for GPU tasks
        )

    # Get the best parameters
    best_params = study.best_params
    best_value = study.best_value
    
    logging.info(f"Best hyperparameters for {split_name}: {best_params}")
    logging.info(f"Best validation MAE: {best_value}")
    
    # Add fixed parameters
    best_params.update({
        # Removed 'objective': 'regression'
        'metric': 'rmse',  # Changed from 'mae' to 'rmse'
        'device_type': 'gpu',
        'gpu_platform_id': 1,
        'boosting_type': 'gbdt',
        'gpu_device_id': 0,
        'n_jobs': 1,
        'random_state': CONFIG["random_seed"],
        'n_estimators': 20000,  # Use full iterations for final model
        'verbose': -1,  # Suppress verbose output
    })
        
            # Save the best parameters
    import json
    with open(params_file, 'w') as f:
        # Save only the parameters that should be persisted (exclude fixed parameters)
        params_to_save = {k: v for k, v in best_params.items() if k in study.best_params}
        json.dump(params_to_save, f, indent=4)
    logging.info(f"Saved best hyperparameters to {params_file}")
    
    return best_params

## Model Training and Evaluation

In [ ]:
def train_and_evaluate_models(df, split_name):
    """Train models on the entire split with optimized hyperparameters"""
    log_step(f"Starting model training for split {split_name}")
    
    # Define path for split-specific metrics
    metrics_file = f"{CONFIG['artifacts_dir']}{split_name}_metrics.csv"
    
    # Check if metrics file already exists
    if os.path.exists(metrics_file):
        logging.info(f"Loading existing metrics for {split_name} from {metrics_file}")
        metrics_df = pd.read_csv(metrics_file)
        
        # Also check for SHAP file
        shap_file = f"{CONFIG['artifacts_dir']}{split_name}_feature_importance.csv"
        if os.path.exists(shap_file):
            shap_df = pd.read_csv(shap_file)
            return metrics_df, shap_df
    
    # Define target and date columns
    target_col = CONFIG['target_column']  # 'qnt' (log-transformed)
    target_orig_col = f"{target_col}_orig"  # Original target for evaluation
    date_col = CONFIG['date_column']
    qnt_max_col = 'qnt_max'
    qnt_max_orig_col = 'qnt_max_orig'
    
    # Exclude target column and date column from features
    all_feature_cols = [col for col in df.columns if col not in [target_col, target_orig_col, date_col] and not col.endswith('_orig')]
    
    # Filter features for baseline model
    baseline_feature_cols, weather_features = filter_weather_features(all_feature_cols)
    
    # Verify weather features aren't in baseline (double-check)
    logging.info(f"Baseline model will use {len(baseline_feature_cols)} features (excluding {len(weather_features)} weather features)")
    
    # Create preprocessors for both models
    full_preprocessor = create_column_transformer(df[all_feature_cols])
    baseline_preprocessor = create_column_transformer(df[baseline_feature_cols])
    
    # Create split for training (80%) and validation (20%)
    df_sorted = df.sort_values(by=date_col)
    split_idx = int(len(df_sorted) * 0.8)
    
    train_df = df_sorted.iloc[:split_idx]
    valid_df = df_sorted.iloc[split_idx:]
    
    # Extract features and target
    X_train_full = train_df[all_feature_cols]
    X_train_baseline = train_df[baseline_feature_cols]
    y_train = train_df[target_col]  # Log-transformed target
    
    X_valid_full = valid_df[all_feature_cols]
    X_valid_baseline = valid_df[baseline_feature_cols]
    y_valid = valid_df[target_col]  # Log-transformed target
    y_valid_orig = valid_df[target_orig_col]  # Original target for evaluation
    qnt_max_valid_orig = valid_df[qnt_max_orig_col]
    
    # Get action prices for stock statistics
    action_price = valid_df['action_price'] if 'action_price' in valid_df.columns else np.ones(len(valid_df))
    
    log_step(f"Train size: {len(y_train)}, Validation size: {len(y_valid)}")
    
    # Preprocess features for full model
    log_step("Preprocessing features for full model")
    X_train_full_proc = full_preprocessor.fit_transform(X_train_full)
    X_valid_full_proc = full_preprocessor.transform(X_valid_full)
    
    # Optimize hyperparameters for full model
    best_params = optimize_hyperparameters(X_train_full_proc, y_train, X_valid_full_proc, y_valid, split_name)
    
    # Clean up memory
    del X_train_full
    gc.collect()
    log_step("After full model preprocessing")
    
    # Preprocess features for baseline model
    log_step("Preprocessing features for baseline model")
    X_train_baseline_proc = baseline_preprocessor.fit_transform(X_train_baseline)
    X_valid_baseline_proc = baseline_preprocessor.transform(X_valid_baseline)
    
    # Clean up memory
    del X_train_baseline
    gc.collect()
    log_step("After baseline preprocessing")
    
    # Results dictionary with all metrics
    results = {
        "split": [],
        "model": [],
        "MAE": [],
        "MAPE": [],
        "sMAPE": [],
        "RMSE": [],
        "R2": [],
        "WAPE": [],
        "Bias": [],
        "over_stock": [],
        "under_stock": [],
        "over_stock_percent": [],
        "under_stock_percent": [],
        "total_sales": [],
        "train_time_sec": [],
        "rows_train": [],
        "rows_valid": []
    }
    
    # Import lightgbm for callbacks
    import lightgbm as lgb
    
    # Define model types and their associated data
    model_configs = {
        "model_with_weather": {
            "file_path": f"{CONFIG['artifacts_dir']}{split_name}_model_with_weather.txt",
            "X_train": X_train_full_proc,
            "X_valid": X_valid_full_proc,
            "preprocessed": True
        },
        "baseline_model": {
            "file_path": f"{CONFIG['artifacts_dir']}{split_name}_baseline_model.txt",
            "X_train": X_train_baseline_proc,
            "X_valid": X_valid_baseline_proc,
            "preprocessed": True
        }
    }
    
    # Train or load each model
    for model_type, config in model_configs.items():
        model_file = config["file_path"]
        X_train_proc = config["X_train"]
        X_valid_proc = config["X_valid"]
        
        if os.path.exists(model_file):
            # Load existing model
            logging.info(f"Loading existing {model_type} from {model_file}")
            model = lgb.Booster(model_file=model_file)
            train_time = 0  # No training time for loaded models
        else:
            # Train new model with optimized hyperparameters
            logging.info(f"Training new {model_type}")
            model_instance = LGBMRegressor(**best_params)
            
            # Set verbosity to -1 to suppress info messages and add GPU settings if not already present
            if 'gpu_platform_id' not in best_params:
                model_instance.set_params(gpu_platform_id=1, gpu_device_id=0, verbose=-1)
            
            start_time = time.time()
            
            model_instance.fit(
                X_train_proc, y_train,
                eval_set=[(X_valid_proc, y_valid)],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=500),
                    lgb.log_evaluation(period=250)
                ]
            )
            
            train_time = time.time() - start_time
            log_step(f"{model_type} training completed in {train_time:.2f}s")
            
            # Save the model
            model_instance.booster_.save_model(model_file)
            model = model_instance.booster_
        
        # Make predictions
        if isinstance(model, lgb.Booster):
            # For loaded Booster models
            y_pred_log = model.predict(X_valid_proc)
        else:
            # For trained LGBMRegressor models
            y_pred_log = model.predict(X_valid_proc)
        
        # Log prediction statistics
        logging.info(f"{model_type} - Log-scale predictions: min={y_pred_log.min():.4f}, max={y_pred_log.max():.4f}, mean={y_pred_log.mean():.4f}")
        
        # Transform back to original scale
        y_pred = np.expm1(y_pred_log)
        
        # Post-process predictions
        y_pred = postprocess_predictions(y_pred, qnt_max_valid_orig.values)
        
        # Calculate metrics
        mae = calculate_mae(y_valid_orig.values, y_pred)
        mape = calculate_mape(y_valid_orig.values, y_pred)
        smape = calculate_smape(y_valid_orig.values, y_pred)
        rmse = calculate_rmse(y_valid_orig.values, y_pred)
        r2 = calculate_r2(y_valid_orig.values, y_pred)
        wape = calculate_wape(y_valid_orig.values, y_pred)
        bias = calculate_bias(y_valid_orig.values, y_pred)
        
        # Calculate stock statistics
        stock_stats = calculate_stock_stats(y_valid_orig.values, y_pred, action_price)
        
        # Log detailed results
        logging.info(f"{model_type} - Training time: {train_time:.2f}s")
        logging.info(f"{model_type} - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
        logging.info(f"{model_type} - MAPE: {mape:.4f}, sMAPE: {smape:.4f}, WAPE: {wape:.4f}, Bias: {bias:.4f}")
        logging.info(f"{model_type} - Over-stock: {stock_stats['over_stock']:.2f} ({stock_stats['over_stock_percent']:.2f}%), Under-stock: {stock_stats['under_stock']:.2f} ({stock_stats['under_stock_percent']:.2f}%)")
        
        # Save results
        results["split"].append(split_name)
        results["model"].append(model_type)
        results["MAE"].append(mae)
        results["MAPE"].append(mape)
        results["sMAPE"].append(smape)
        results["RMSE"].append(rmse)
        results["R2"].append(r2)
        results["WAPE"].append(wape)
        results["Bias"].append(bias)
        results["over_stock"].append(stock_stats['over_stock'])
        results["under_stock"].append(stock_stats['under_stock'])
        results["over_stock_percent"].append(stock_stats['over_stock_percent'])
        results["under_stock_percent"].append(stock_stats['under_stock_percent'])
        results["total_sales"].append(stock_stats['total_sales'])
        results["train_time_sec"].append(train_time)
        results["rows_train"].append(len(y_train))
        results["rows_valid"].append(len(y_valid))
    
    # Calculate SHAP values for feature importance
    log_step("Calculating SHAP values for feature importance")
    
    # Load the models again for SHAP calculation if needed
    model_with_weather = None
    baseline_model = None
    
    # Load or use model_with_weather
    weather_model_file = f"{CONFIG['artifacts_dir']}{split_name}_model_with_weather.txt"
    if os.path.exists(weather_model_file):
        try:
            model_with_weather = lgb.Booster(model_file=weather_model_file)
        except Exception as e:
            logging.error(f"Error loading model_with_weather for SHAP: {e}")
    
    # Load or use baseline_model
    baseline_model_file = f"{CONFIG['artifacts_dir']}{split_name}_baseline_model.txt"
    if os.path.exists(baseline_model_file):
        try:
            baseline_model = lgb.Booster(model_file=baseline_model_file)
        except Exception as e:
            logging.error(f"Error loading baseline_model for SHAP: {e}")
    
    # Skip SHAP calculation if models couldn't be loaded
    if model_with_weather is None or baseline_model is None:
        logging.warning("Skipping SHAP calculation due to missing models")
        feature_importance_data = pd.DataFrame({
            "split": [], "model_type": [], "feature": [], "importance_pct": []
        })
    else:
        feature_importance_data = calculate_feature_importance(
            model_with_weather, baseline_model,
            X_train_full_proc, X_train_baseline_proc, 
            full_preprocessor, baseline_preprocessor,
            split_name
        )
    
    # Create metrics DataFrame
    metrics_df = pd.DataFrame(results)
    
    # Save metrics for this split
    metrics_df.to_csv(metrics_file, index=False)
    logging.info(f"Saved metrics for {split_name} to {metrics_file}")
    
    # Clean up memory
    del X_train_full_proc, X_valid_full_proc, X_train_baseline_proc, X_valid_baseline_proc
    gc.collect()
    
    return metrics_df, feature_importance_data

In [ ]:
def calculate_feature_importance(model_with_weather, baseline_model, 
                                X_train_full, X_train_baseline,
                                full_preprocessor, baseline_preprocessor,
                                split_name):
    """Calculate SHAP feature importance for both models"""
    # Check if SHAP results already exist for this split
    shap_file = f"{CONFIG['artifacts_dir']}{split_name}_feature_importance.csv"
    
    if os.path.exists(shap_file):
        logging.info(f"Loading existing SHAP feature importance from {shap_file}")
        return pd.read_csv(shap_file)
    
    # Initialize feature importance data
    feature_importance_data = {
        "split": [],
        "model_type": [],
        "feature": [],
        "importance_pct": []
    }
    
    try:
        # Extract feature names from preprocessor for full model
        feature_names_full = []
        for name, transformer, columns in full_preprocessor.transformers_:
            if name == 'low_cat' and hasattr(transformer, 'get_feature_names_out'):
                cat_features = transformer.get_feature_names_out(columns)
                feature_names_full.extend(cat_features)
            elif name in ['bool', 'num', 'med_cat']:
                feature_names_full.extend(columns)
        
        # Extract feature names for baseline model
        feature_names_baseline = []
        for name, transformer, columns in baseline_preprocessor.transformers_:
            if name == 'low_cat' and hasattr(transformer, 'get_feature_names_out'):
                cat_features = transformer.get_feature_names_out(columns)
                feature_names_baseline.extend(cat_features)
            elif name in ['bool', 'num', 'med_cat']:
                feature_names_baseline.extend(columns)
        
        # Calculate SHAP values for both models
        models_to_analyze = {
            "model_with_weather": (model_with_weather, X_train_full, feature_names_full),
            "baseline_model": (baseline_model, X_train_baseline, feature_names_baseline)
        }
        
        for model_type, (model, X, feature_names) in models_to_analyze.items():
            # Sample dataset to a more moderate size that balances accuracy and performance
            # For tree-based models like LightGBM, 10,000 samples is typically sufficient
            max_samples = 10000  # Reduced from 100,000 for better efficiency
            
            if X.shape[0] > max_samples:
                logging.info(f"Sampling {max_samples} rows from {X.shape[0]} for SHAP calculation")
                sample_indices = np.random.choice(X.shape[0], max_samples, replace=False)
                X_sample = X[sample_indices]
            else:
                X_sample = X
                logging.info(f"Using all {X.shape[0]} rows for SHAP calculation")
            
            # Convert sparse matrix to dense for SHAP calculation
            if scipy.sparse.issparse(X_sample):
                logging.info(f"Converting sparse matrix to dense for SHAP calculation")
                X_sample = X_sample.toarray()
            
            log_step(f"Calculating SHAP values for {model_type} with {X_sample.shape[0]} samples")
            
            # Create SHAP explainer - handling both LGBMRegressor and Booster objects
            import lightgbm as lgb
            if isinstance(model, lgb.Booster):
                explainer = shap.TreeExplainer(model)
            else:
                explainer = shap.TreeExplainer(model)
            
            shap_values = explainer.shap_values(X_sample)
            
            # Calculate feature importance
            importance_values = np.abs(shap_values).mean(0)
            importance_sum = importance_values.sum()
            
            # Convert to percentages and create DataFrame
            if importance_sum > 0:  # Avoid division by zero
                for i, feature_name in enumerate(feature_names[:len(importance_values)]):
                    importance_pct = (importance_values[i] / importance_sum) * 100
                    
                    feature_importance_data["split"].append(split_name)
                    feature_importance_data["model_type"].append(model_type)
                    feature_importance_data["feature"].append(feature_name)
                    feature_importance_data["importance_pct"].append(importance_pct)
                    
            # Clean up memory
            del explainer, shap_values, X_sample
            gc.collect()
            log_step(f"Completed SHAP analysis for {model_type}")
    
    except Exception as e:
        logging.error(f"Error calculating feature importance: {e}")
    
    # Create DataFrame from the results
    feature_importance_df = pd.DataFrame(feature_importance_data)
    
    # Save to file
    if not feature_importance_df.empty:
        feature_importance_df.to_csv(shap_file, index=False)
        logging.info(f"Saved SHAP feature importance to {shap_file}")
    
    return feature_importance_df

## Main Execution

In [ ]:
# Find all split files
split_files = glob(CONFIG['splits_dir'] + 'sales_category*.csv')

# Initialize results tracking
all_results = []
all_feature_importance = []

# First, check for existing results and feature importance files
for split_file in glob(CONFIG['splits_dir'] + 'sales_category*.csv'):
    split_name = os.path.basename(split_file).replace('sales_category_', '').replace('.csv', '')
    
    # Check for existing feature importance files
    shap_file = f"{CONFIG['artifacts_dir']}{split_name}_feature_importance.csv"
    metrics_file = f"{CONFIG['artifacts_dir']}{split_name}_metrics.csv"
    
    # Load existing SHAP results if available
    if os.path.exists(shap_file):
        logging.info(f"Found existing SHAP results for {split_name}")
        try:
            shap_data = pd.read_csv(shap_file)
            all_feature_importance.append(shap_data)
        except Exception as e:
            logging.error(f"Error loading SHAP file for {split_name}: {str(e)}")
    
    # Load existing metrics if available
    if os.path.exists(metrics_file):
        logging.info(f"Found existing metrics for {split_name}")
        try:
            metrics_data = pd.read_csv(metrics_file)
            all_results.append(metrics_data)
        except Exception as e:
            logging.error(f"Error loading metrics file for {split_name}: {str(e)}")

# Process each split file
for split_file in tqdm(split_files, desc="Processing splits"):
    split_name = os.path.basename(split_file).replace('sales_category_', '').replace('.csv', '')
    
    # Check if we already have both metrics and SHAP results for this split
    metrics_file = f"{CONFIG['artifacts_dir']}{split_name}_metrics.csv"
    shap_file = f"{CONFIG['artifacts_dir']}{split_name}_feature_importance.csv"
    
    if os.path.exists(metrics_file) and os.path.exists(shap_file):
        logging.info(f"Skipping split {split_name} - results already exist")
        continue
    
    # Process each split
    log_step(f"Starting processing of split {split_name}")
    
    # Load and preprocess data
    df = load_and_preprocess_data(split_file)
    
    # Train and evaluate models
    try:
        split_results, feature_importance = train_and_evaluate_models(df, split_name)
        
        # Save split-specific metrics
        if not split_results.empty:
            split_results.to_csv(metrics_file, index=False)
            logging.info(f"Saved metrics for {split_name} to {metrics_file}")
        
        all_results.append(split_results)
        all_feature_importance.append(feature_importance)
    except Exception as e:
        logging.error(f"Error processing split {split_name}: {str(e)}")
        continue
    
    # Clean up to free memory
    del df, split_results, feature_importance
    gc.collect()
    mem_mb = get_memory_usage()
    logging.info(f'After split {split_name} completion - memory: {mem_mb} MB')

# Check if we have any results
if len(all_results) == 0:
    logging.info("No results were collected. Check if all files already exist.")
    
    # Try to load previously saved aggregate results
    try:
        results_df = pd.read_csv(f"{CONFIG['artifacts_dir']}experiment_metrics.csv")
        feature_importance_df = pd.read_csv(f"{CONFIG['artifacts_dir']}feature_importance.csv")
        
        # Display previously saved results
        print("\nLoaded Previously Saved Results:")
        print("\nPrimary Metrics:")
        display(results_df[['split', 'model', 'MAE', 'RMSE', 'R2', 'sMAPE', 'WAPE']])
        
        print("\nStock Impact Metrics:")
        display(results_df[['split', 'model', 'over_stock_percent', 'under_stock_percent', 'Bias']])
    except Exception as load_err:
        logging.error(f"Error loading previous results: {load_err}")
        print("No results were collected and no previous results could be loaded.")
else:
    # Combine results
    results_df = pd.concat(all_results)
    feature_importance_df = pd.concat(all_feature_importance)

    # Calculate averages across splits for each model, including ALL metrics
    avg_results = results_df.groupby('model').agg({
        'MAE': 'mean',
        'MAPE': 'mean', 
        'sMAPE': 'mean',
        'RMSE': 'mean',
        'R2': 'mean',
        'WAPE': 'mean',
        'Bias': 'mean',
        'over_stock': 'mean',
        'under_stock': 'mean',
        'over_stock_percent': 'mean',
        'under_stock_percent': 'mean',
        'total_sales': 'mean',
        'train_time_sec': 'mean',
        'rows_train': 'mean',
        'rows_valid': 'mean'
    }).reset_index()
    avg_results['split'] = 'AVERAGE'

    # Add averages to results
    final_results = pd.concat([results_df, avg_results])

    # Save aggregate results
    final_results.to_csv(f"{CONFIG['artifacts_dir']}experiment_metrics.csv", index=False)

    # Save aggregate feature importance
    feature_importance_df.to_csv(f"{CONFIG['artifacts_dir']}feature_importance.csv", index=False)

    # Display results
    print("\nFinal Results:")
    print("\nPrimary Metrics:")
    display(final_results[['split', 'model', 'MAE', 'RMSE', 'R2', 'sMAPE', 'WAPE']])

    print("\nStock Impact Metrics:")
    display(final_results[['split', 'model', 'over_stock_percent', 'under_stock_percent', 'Bias']])

    print("\nDetailed Metrics:")
    display(final_results)

In [ ]:
# Calculate the total influence of weather features
all_features = feature_importance_df[feature_importance_df['model_type'] == 'model_with_weather']

# Identify weather features
weather_features = []
for feature in all_features['feature'].unique():
    if any(re.search(pattern, feature) for pattern in WEATHER_PATTERNS):
        weather_features.append(feature)

# Calculate total importance of weather features
weather_importance = all_features[all_features['feature'].isin(weather_features)]['importance_pct'].sum()
total_importance = all_features['importance_pct'].sum()
weather_percentage = (weather_importance / total_importance) * 100

print(f"\nWeather Features Impact:")
print(f"Weather features account for {weather_percentage:.2f}% of total feature importance in the model with weather")
print(f"Number of weather features used: {len(weather_features)} out of {len(all_features['feature'].unique())} total features")

# Агрегируем важность (например, суммой) по feature
agg_features = all_features.groupby('feature', as_index=False)['importance_pct'].mean()

# Сортируем по убыванию суммарной важности
top_weather_features = agg_features.sort_values('importance_pct', ascending=False)

print("\nTop 10 most important weather features:")
display(top_weather_features[['feature', 'importance_pct']])

# Сохраняем в CSV (если нужно больше — поменяй head(10) на head(10000) или сколько нужно)
top_weather_features[['feature', 'importance_pct']].to_csv("results.csv", index=False)